In [2]:
! pip install -U accelerate
! pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 56.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 105.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 30.5 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.18.0
    Uninstalling huggingface-hub-0.18.0:
      Successfully uninstalled huggingface-hub-0.18.0


In [3]:
import pandas as pd
import torch
import re

from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import TextDataset,DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments, AutoModelWithLMHead, AutoModelForCausalLM, GPT2LMHeadModel
from transformers import pipeline

from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#Подготовка данных и обучение модели

In [ ]:
#загружаем данные
data = pd.read_csv('/content/drive/MyDrive/Innopolis/итоговая аттестация/jokes.csv')
data

,theme,text,rating
0,pro-sudey,На суде в Стамбуле обвиняемый сказал:\r\n- На...,5
1,pro-sudey,"- Вы продолжаете утверждать, что обвиняемый н...",4
2,pro-sudey,"На суде.\r\n- Итак, когда дело дошло до столкн...",0
3,pro-sudey,Старую леди сбил автомобиль. На суде ее спраши...,4
4,pro-sudey,"Судья говорит:\r\n- Согласно вашей жалобе, об...",2
...,...,...,...
130199,raznie,"Збежал медведь из зоопарка, ну передали по рад...",0
130200,raznie,"Разговаривают два грузина, Гоги и Авас:\r\nГ: ...",0
130201,raznie,В каждом из нас спит гений и с каждым днем вс...,0
130202,raznie,Очередь... Последней в ней стоит бабка - гор...,0


In [ ]:
train, test = train_test_split(data['text'],test_size=0.15)
train

46420     1896 Американский миллионер приехал в Лондон ...
23908     - Что надо делать, если с эпилептиком случился...
129631    Вышка управления вызывает борт 762. Не можем с...
30984     Человек, который (хотя бы) на мгновение был в...
74953     Іспит, студентка валиться безповоротно. За две...
                                ...                        
74254     LandGraf\r\nСтою ночью на балконе курю, 8 этаж...
99340     Генерал обходит казармы. Заходит в одну, потяг...
51844     Все компьютерщики Питера выражают огромную бла...
25243     Поймали дикие племена троих: американца, фран...
16182     Американский анекдот\r\n- Кто управляет машин...
Name: text, Length: 110673, dtype: object

In [ ]:
#записываем данные в .txt файлы и убираем пуслые строки
with open('train_dataset.txt', "w") as my_output_file:
   [ my_output_file.write("".join(re.sub(r"\r\n", " ", row))+'\n') for row in train]
   my_output_file.close()

In [ ]:
with open('test_dataset.txt', "w") as my_output_file:
   [ my_output_file.write("".join(re.sub(r"\r\n", " ", row))+'\n') for row in test]
   my_output_file.close()

In [ ]:
#далее загружаем токенизатор русской модели gpt2

tokenizer = AutoTokenizer.from_pretrained("Nehc/gpt2_lovecraft_ru")

train_path = 'train_dataset.txt'
test_path = 'test_dataset.txt'

In [ ]:
def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)

    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

train_dataset,test_dataset,data_collator = load_dataset(train_path,test_path,tokenizer)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [ ]:
#в TrainingArguments определяем гиперпараметры

model = AutoModelWithLMHead.from_pretrained("Nehc/gpt2_lovecraft_ru")

training_args = TrainingArguments(
    output_dir="./gpt2_lovecraft_ru",
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    eval_steps = 400,
    save_steps=800,
    warmup_steps=500,
    prediction_loss_only=True,
    )

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1499: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [ ]:
trainer.train()

In [ ]:
trainer.save_model('/content/drive/MyDrive/Innopolis/итоговая аттестация/Model')

#Тестирование модели

In [ ]:
#используем pipeline для генерации текста

chef = pipeline('text-generation',
                model='/content/drive/MyDrive/Innopolis/итоговая аттестация/Model', #укажите свой путь к папке Model на диске
                tokenizer='Nehc/gpt2_lovecraft_ru',
                )

In [ ]:
result = chef('пошла блондинка в магазин и', do_sample=True, max_new_tokens=100)
result[0]['generated_text'].split('\n')[0]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'пошла блондинка в магазин и купила себе шубы в горошек. Приходит к мужу и говорит: - Муж мой купил яйца в горошек и вобще у меня не так голова болит! Муж: - Да нет! Не так голова болит! '

#Настройка окружения для запуска телеграм-бота

In [4]:
%env PYTHONPATH = # /env/python
!wget https://repo.anaconda.com/miniconda/Miniconda3-py38_4.12.0-Linux-x86_64.sh
!chmod +x Miniconda3-py38_4.12.0-Linux-x86_64.sh
!./Miniconda3-py38_4.12.0-Linux-x86_64.sh -b -f -p /usr/local
!conda update conda

env: PYTHONPATH=# /env/python
--2023-10-23 17:44:28--  https://repo.anaconda.com/miniconda/Miniconda3-py38_4.12.0-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 76120962 (73M) [application/x-sh]
Saving to: ‘Miniconda3-py38_4.12.0-Linux-x86_64.sh’

Miniconda3-py38_4.1 100%[===================>]  72.59M   209MB/s    in 0.3s    

2023-10-23 17:44:28 (209 MB/s) - ‘Miniconda3-py38_4.12.0-Linux-x86_64.sh’ saved [76120962/76120962]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: | / - done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - _openmp_mutex==4.5=1_gnu
    - brotlipy==0.7.0=py38h27cfd23_1003
    - ca-certificates==2022.3.29=h06a4308_1
    - certifi==2021.10.8=py38h06a4308_2
    - cffi==

In [5]:
import sys
sys.path.append('/usr/local/lib/python3.8/site-packages')

In [6]:
!conda create -n myenv python=3.9

Solving environment: \ done


==> WARNING: A newer version of conda exists. <==
  current version: 4.12.0
  latest version: 23.9.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /usr/local/envs/myenv

  added / updated specs:
    - python=3.9


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    pip-23.3                   |   py39h06a4308_0         2.6 MB
    python-3.9.18              |       h955ad1f_0        25.1 MB
    setuptools-68.0.0          |   py39h06a4308_0         928 KB
    tzdata-2023c               |       h04d1e81_0         116 KB
    wheel-0.41.2               |   py39h06a4308_0         108 KB
    ------------------------------------------------------------
                                           Total:        28.8 MB

The following NEW packages will be INSTALLED:

  _libgcc_mutex      pkgs/ma

In [7]:
%%shell
eval "$(conda shell.bash hook)"
conda activate myenv
pip install -U transformers
pip install -U torch
pip install aiogram==2.25.1
export PYTHONIOENCODING=UTF-8
pip install protobuf==3.20.*

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.5/121.5 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 99.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 738.9/738.9 kB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 773.3/773.3 kB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 99.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.

In [8]:
import shutil
#укажите здесь свой путь к файлу bot.py на диске
shutil.copy('/content/drive/MyDrive/Innopolis/итоговая аттестация/bot.py', '/content')

'/content/bot.py'

In [ ]:
%%shell
eval "$(conda shell.bash hook)"
conda activate myenv
python /content/bot.py

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
